## Imports

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler


##wilcoxin rank-sum test
from scipy.stats import mannwhitneyu

from gtda.homology import VietorisRipsPersistence
from gtda.plotting import plot_point_cloud

from gtda.mapper import (
        CubicalCover,
    make_mapper_pipeline,
    Projection,
    plot_static_mapper_graph,
    plot_interactive_mapper_graph
)
import umap
#pip install umap-learn
from sklearn.cluster import DBSCAN, KMeans

from sklearn.decomposition import PCA

import plotly.express as px

In [63]:
data_raw = pd.read_csv("Mice Protein Expression.csv")

In [64]:
data_raw

,MouseID,DYRK1A_N,ITSN1_N,BDNF_N,NR1_N,NR2A_N,pAKT_N,pBRAF_N,pCAMKII_N,pCREB_N,...,pCFOS_N,SYP_N,H3AcK18_N,EGR1_N,H3MeK4_N,CaNA_N,Genotype,Treatment,Behavior,class
0,309_1,0.503644,0.747193,0.430175,2.816329,5.990152,0.218830,0.177565,2.373744,0.232224,...,0.108336,0.427099,0.114783,0.131790,0.128186,1.675652,Control,Memantine,C/S,c-CS-m
1,309_2,0.514617,0.689064,0.411770,2.789514,5.685038,0.211636,0.172817,2.292150,0.226972,...,0.104315,0.441581,0.111974,0.135103,0.131119,1.743610,Control,Memantine,C/S,c-CS-m
2,309_3,0.509183,0.730247,0.418309,2.687201,5.622059,0.209011,0.175722,2.283337,0.230247,...,0.106219,0.435777,0.111883,0.133362,0.127431,1.926427,Control,Memantine,C/S,c-CS-m
3,309_4,0.442107,0.617076,0.358626,2.466947,4.979503,0.222886,0.176463,2.152301,0.207004,...,0.111262,0.391691,0.130405,0.147444,0.146901,1.700563,Control,Memantine,C/S,c-CS-m
4,309_5,0.434940,0.617430,0.358802,2.365785,4.718679,0.213106,0.173627,2.134014,0.192158,...,0.110694,0.434154,0.118481,0.140314,0.148380,1.839730,Control,Memantine,C/S,c-CS-m
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1075,J3295_11,0.254860,0.463591,0.254860,2.092082,2.600035,0.211736,0.171262,2.483740,0.207317,...,0.183324,0.374088,0.318782,0.204660,0.328327,1.364823,Ts65Dn,Saline,S/C,t-SC-s
1076,J3295_12,0.272198,0.474163,0.251638,2.161390,2.801492,0.251274,0.182496,2.512737,0.216339,...,0.175674,0.375259,0.325639,0.200415,0.293435,1.364478,Ts65Dn,Saline,S/C,t-SC-s
1077,J3295_13,0.228700,0.395179,0.234118,1.733184,2.220852,0.220665,0.161435,1.989723,0.185164,...,0.158296,0.422121,0.321306,0.229193,0.355213,1.430825,Ts65Dn,Saline,S/C,t-SC-s
1078,J3295_14,0.221242,0.412894,0.243974,1.876347,2.384088,0.208897,0.173623,2.086028,0.192044,...,0.196296,0.397676,0.335936,0.251317,0.365353,1.404031,Ts65Dn,Saline,S/C,t-SC-s



## Analysis

The analysis is structured in the following steps:

1. Data Preprocessing

We fill in the missing protein values with the class averages.

We normalise the data with the min_max_scaler()

2. The Shape of the Dataset

We compute the persistent homology of the data to better have an idea of the topological features that are present in the data

3. Naive Wilcoxon Approach

We use the Wilcoxon test to differentiat the protein distribution amongst the different classes. This technique will highlight how protein expression varies amongst the different classes. Unfortunately, due to the individuality of each mouse, the results are too noisy to conclude.

4. Application of the Mapper Algorithm

We apply the Mapper algorithm and try to relate its shape with a finer decomposition of the classes. The purpose would be to refine the classes and study the difference in protein expressions not amongst single individuals but rather amongst regions of the mapper graph. This will remove noise due to individuality and allow us to better identify the critical proteins. We apply again the Wilcoxon test to quantify the significance of protein expressions amongst different regions of the Mapper graph.

In [65]:
data_class_c_CS_m = data_raw[data_raw["class"]=="c-CS-m"].fillna(data_raw.mean())
data_class_c_CS_s = data_raw[data_raw["class"]=="c-CS-s"].fillna(data_raw.mean())
data_class_c_SC_m = data_raw[data_raw["class"]=="c-SC-m"].fillna(data_raw.mean())
data_class_c_SC_s = data_raw[data_raw["class"]=="c-SC-s"].fillna(data_raw.mean())
data_class_t_CS_m = data_raw[data_raw["class"]=="t-CS-m"].fillna(data_raw.mean())
data_class_t_CS_s = data_raw[data_raw["class"]=="t-CS-s"].fillna(data_raw.mean())
data_class_t_SC_m = data_raw[data_raw["class"]=="t-SC-m"].fillna(data_raw.mean())
data_class_t_SC_s = data_raw[data_raw["class"]=="t-SC-s"].fillna(data_raw.mean())

data_temp = data_class_c_CS_m.copy()

# put together the whole dataset with filled classes (un-normalized)
data = data_temp.append(data_class_c_CS_s, ignore_index = True).append(data_class_c_SC_s, ignore_index = True).append(data_class_t_CS_s, ignore_index = True).append(data_class_t_SC_s, ignore_index = True).append(data_class_t_CS_m, ignore_index = True).append(data_class_t_SC_m, ignore_index = True).append(data_class_c_SC_m, ignore_index = True)

# extract only numerical values describing the gene expressions
data_num = data.loc[:,"DYRK1A_N":"CaNA_N"]

# normalise the dataset
x = data_num.values 
min_max_scaler = MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)

C:\Users\ragha\AppData\Local\Temp/ipykernel_16620/919497570.py:1: FutureWarning:

Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.

C:\Users\ragha\AppData\Local\Temp/ipykernel_16620/919497570.py:2: FutureWarning:

Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.

C:\Users\ragha\AppData\Local\Temp/ipykernel_16620/919497570.py:3: FutureWarning:

Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.

C:\Users\ragha\AppData\Local\Temp/ipykernel_16620/919497570.py:4: FutureWarning:

Dropping of nuisance columns in DataFrame reductions (with 'numeric_only

In [66]:
x_scaled


array([[0.1511224 , 0.21288505, 0.82463786, ..., 0.10288975, 0.08457952,
        0.70573764],
       [0.1557504 , 0.18822566, 0.77645463, ..., 0.11587379, 0.09397699,
        0.74977105],
       [0.15345859, 0.20569615, 0.79357192, ..., 0.10904993, 0.08216206,
        0.86822857],
       ...,
       [0.04769956, 0.08163664, 0.43138051, ..., 0.84008175, 0.33209664,
        0.52761843],
       [0.06311134, 0.1074225 , 0.52841589, ..., 0.82154126, 0.33209664,
        0.50263706],
       [0.06074481, 0.08217765, 0.5852416 , ..., 0.9078772 , 0.33209664,
        0.4885845 ]])

In [67]:
data['MouseNumber'] = data.MouseID.apply(lambda x: x.split('_')[0])
data['MouseVersion'] = data.MouseID.apply(lambda x: x.split('_')[1])
data.drop(['MouseID'], axis=1, inplace=True)

In [68]:
data

,DYRK1A_N,ITSN1_N,BDNF_N,NR1_N,NR2A_N,pAKT_N,pBRAF_N,pCAMKII_N,pCREB_N,pELK_N,...,H3AcK18_N,EGR1_N,H3MeK4_N,CaNA_N,Genotype,Treatment,Behavior,class,MouseNumber,MouseVersion
0,0.503644,0.747193,0.430175,2.816329,5.990152,0.218830,0.177565,2.373744,0.232224,1.750936,...,0.114783,0.131790,0.128186,1.675652,Control,Memantine,C/S,c-CS-m,309,1
1,0.514617,0.689064,0.411770,2.789514,5.685038,0.211636,0.172817,2.292150,0.226972,1.596377,...,0.111974,0.135103,0.131119,1.743610,Control,Memantine,C/S,c-CS-m,309,2
2,0.509183,0.730247,0.418309,2.687201,5.622059,0.209011,0.175722,2.283337,0.230247,1.561316,...,0.111883,0.133362,0.127431,1.926427,Control,Memantine,C/S,c-CS-m,309,3
3,0.442107,0.617076,0.358626,2.466947,4.979503,0.222886,0.176463,2.152301,0.207004,1.595086,...,0.130405,0.147444,0.146901,1.700563,Control,Memantine,C/S,c-CS-m,309,4
4,0.434940,0.617430,0.358802,2.365785,4.718679,0.213106,0.173627,2.134014,0.192158,1.504230,...,0.118481,0.140314,0.148380,1.839730,Control,Memantine,C/S,c-CS-m,309,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1075,0.253757,0.461571,0.291971,2.126878,3.044654,0.252469,0.196222,3.901889,0.202662,1.259983,...,0.169609,0.252994,0.205440,1.275132,Control,Memantine,S/C,c-SC-m,365,11
1076,0.249052,0.444796,0.288243,2.078382,3.093552,0.249263,0.194480,3.821113,0.198483,1.289718,...,0.169609,0.250447,0.205440,1.261456,Control,Memantine,S/C,c-SC-m,365,12
1077,0.258424,0.437801,0.279959,1.966810,2.828477,0.281986,0.204966,3.436281,0.209020,1.196858,...,0.169609,0.319888,0.205440,1.400759,Control,Memantine,S/C,c-SC-m,365,13
1078,0.294966,0.498586,0.317025,2.208710,3.060238,0.294400,0.240950,3.947115,0.231052,1.424208,...,0.169609,0.315157,0.205440,1.362205,Control,Memantine,S/C,c-SC-m,365,14


In [69]:
df = pd.DataFrame(x_scaled, columns = data_num.columns)

# add the non numerical classes
df["MouseNumber"] = data.MouseNumber
df["MouseVersion"] = data.MouseVersion
df["Genotype"] = data.Genotype
df["Treatment"] = data.Treatment
df["Behavior"] = data.Behavior
df["class"] = data["class"].values

df

,DYRK1A_N,ITSN1_N,BDNF_N,NR1_N,NR2A_N,pAKT_N,pBRAF_N,pCAMKII_N,pCREB_N,pELK_N,...,H3AcK18_N,EGR1_N,H3MeK4_N,CaNA_N,MouseNumber,MouseVersion,Genotype,Treatment,Behavior,class
0,0.151122,0.212885,0.824638,0.612119,0.630482,0.327006,0.448666,0.168257,0.617322,0.232553,...,0.087715,0.102890,0.084580,0.705738,309,1,Control,Memantine,C/S,c-CS-m
1,0.155750,0.188226,0.776455,0.601070,0.585247,0.311887,0.429899,0.154925,0.590173,0.205362,...,0.080692,0.115874,0.093977,0.749771,309,2,Control,Memantine,C/S,c-CS-m
2,0.153459,0.205696,0.793572,0.558911,0.575910,0.306369,0.441381,0.153485,0.607102,0.199194,...,0.080465,0.109050,0.082162,0.868229,309,3,Control,Memantine,C/S,c-CS-m
3,0.125169,0.157688,0.637326,0.468152,0.480646,0.335530,0.444307,0.132074,0.486945,0.205135,...,0.126763,0.164241,0.144543,0.721879,309,4,Control,Memantine,C/S,c-CS-m
4,0.122146,0.157838,0.637787,0.426467,0.441977,0.314976,0.433100,0.129086,0.410194,0.189152,...,0.096959,0.136298,0.149281,0.812053,309,5,Control,Memantine,C/S,c-CS-m
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1075,0.045731,0.091720,0.462826,0.328022,0.193790,0.397703,0.522399,0.417951,0.464498,0.146183,...,0.224755,0.577909,0.332097,0.446218,365,11,Control,Memantine,S/C,c-SC-m
1076,0.043747,0.084604,0.453066,0.308039,0.201039,0.390965,0.515514,0.404752,0.442893,0.151414,...,0.224755,0.567930,0.332097,0.437356,365,12,Control,Memantine,S/C,c-SC-m
1077,0.047700,0.081637,0.431381,0.262064,0.161740,0.459739,0.556958,0.341872,0.497363,0.135078,...,0.224755,0.840082,0.332097,0.527618,365,13,Control,Memantine,S/C,c-SC-m
1078,0.063111,0.107423,0.528416,0.361742,0.196100,0.485829,0.699176,0.425341,0.611265,0.175074,...,0.224755,0.821541,0.332097,0.502637,365,14,Control,Memantine,S/C,c-SC-m


In [70]:
y =x_scaled
y
print(y.shape)

(1080, 77)


In [71]:
y = y.reshape(1,*y.shape)
print(y.shape)

(1, 1080, 77)


### Abstract Topological Information
By using Vietoris-Rips algo. to understand the shape of the dataset i.e. by persistent homology shape is determined

In [72]:
vr = VietorisRipsPersistence(collapse_edges=True, homology_dimensions=[0,1], n_jobs=-1)

vr.fit_transform_plot(x_scaled.reshape(1, *x_scaled.shape))

array([[[0.        , 0.1479542 , 0.        ],
        [0.        , 0.17491628, 0.        ],
        [0.        , 0.17524096, 0.        ],
        ...,
        [0.3617591 , 0.37778676, 1.        ],
        [0.35509679, 0.44061026, 1.        ],
        [0.35249808, 0.37803122, 1.        ]]])

This diagram shows us, abstractly, what the shape of our point cloud looks like. In particular, when looking at the H0 component (i.e. the red points), we see (left) that there are around six points that are separated from the others. These are reasonably clusters connected components. On the other hand, there are no relevant 1-loops in the dataset, as demonstrated by the H1 component (i.e. the noisy green points, close to the diagonal of the diagram).

In [73]:
pca = PCA(n_components=3)
data_red = pca.fit_transform(x_scaled)

df_pca = pd.DataFrame(data_red, columns = ["x", "y", "z"])
px.scatter_3d(df_pca, x="x", y="y", z="z")

Nothoing significant we can say from the plot

In [74]:
import umap.umap_ as umap

In [75]:
reducer = umap.UMAP(n_components=3, n_neighbors=15, random_state=42)
embedding = reducer.fit_transform(x_scaled)
print("Shape of the reduced dataset: ", embedding.shape)

df_umap = pd.DataFrame(embedding, columns = ["x", "y", "z"])
px.scatter_3d(df_umap, x="x", y="y", z="z")

Shape of the reduced dataset:  (1080, 3)


The VR algorithms suggests that there are 6 clusters that are reasonably separated from the others. Amongst these 6, three of them are very separated from the rest. While from the PCA plot we do not see any kind of structure, from the UMAP one we clearly see the three separate clusters, 2 small ones and a very large one. The large cluster can also be split into subclusters, we can for sure see at least three of them.

Concerning the number of loops, no evident loop appear in the UMAP plot, which is consistent with the persistence diagram above.

We can initially conclude on how superior the topological dimensionality reduction is to maintain the shape of the high dimensional point cloud compared with a standard linear algorithm.

## Wilcoxin Approach
To compare two distributions to check like which protein is playing role in when memantine is takin we can test with t-CS-m with t-CS-s , similarly we can compare whichi proteins playin their part in learninf  by -c-CS-s and c-SC-s 

In [76]:

# get all proteins names
proteins = data_num.columns

# extract p-values for the t-CS-m and t-CS-s comparison
p_values = []
for protein in proteins:
    w, p = mannwhitneyu(x=data_class_t_CS_m[protein], y=data_class_t_CS_s[protein])
    p_values.append(p)
    
px.histogram(p_values, nbins=20)

In [77]:

# extract p-values for the c-CS-s and c-SC-s comparison
p_values = []
for protein in proteins:
    w, p = mannwhitneyu(x=data_class_c_CS_s[protein], y=data_class_c_SC_s[protein])
    p_values.append(p)
    
px.histogram(p_values, nbins=20)

Analysing the p-values resulting from the Wilcoxon test and we look for significative differences amongst protein expressions between pairs of different classes.

#### Observation
In both cases, far too many proteins have been identified to be significant. This is most likely due to the effect of the individuality of each mouse in protein expression.

We thus proceed with a more refined -- unsupervised!! -- splitting of the classes that takes individuality into account.

## Application of Mapper Algo 

Intution is to compare between cluster not amongst individuals of a class.

### In 3D PCA plot


In [78]:
def class2num(string):
    c, cs, m = string.split("-")
    return (c=='c') + (cs=='CS')*5 + (m=='m')*10

color_list = []
for cls in df["class"].values:
    color_list.append(class2num(cls))

color_array = np.array(color_list)

In [79]:
px.scatter_3d(df_pca, x="x", y="y", z="z", color=color_array)


In [80]:
px.scatter_3d(df_umap, x="x", y="y", z="z", color=color_array)


### Mapper algo 
mapper algo with a custom filter function based on UMAP embedding 


In [81]:
#cubical covering over 3D -UMAP

def filter_func(row):
    ind = np.where(np.all(x_scaled==row,axis=1))
    return embedding[ind][0]
    #return reducer.transform(row.reshape(1, -1))[0]

# Define cover
cover = CubicalCover(n_intervals=25, overlap_frac=0.3)

# Choose clustering algorithm – default is DBSCAN
clusterer = DBSCAN()

# Configure parallelism of clustering step
n_jobs = -1

# Initialise pipeline
pipe = make_mapper_pipeline(
    filter_func=filter_func,
    cover=cover,
    clusterer=clusterer,
    verbose=False,
    n_jobs=n_jobs,
    store_edge_elements=True,
    contract_nodes=True
    )

In [117]:
fig = plot_static_mapper_graph(pipe, x_scaled,
                               layout_dim=3,
                               node_scale=40,
                               layout="fruchterman_reingold",
                               color_data=color_array)
fig.show(config={'scrollZoom': True})

#### Now wilcoxon tests among the mapper clusters

mapper cluster is basically the connected region of the mapper graph.
then put wilcoxon test on the t-CS-m <> 15, t-CS-s<> 5 and c-CS-s <> 6, c-SC-s <> 1 to get the learnig on specific proteins.



In [106]:
mapper_graph = pipe.fit_transform(x_scaled)
nodes_elements = mapper_graph.vs["node_elements"]
edges_elements = mapper_graph.es["edge_elements"]
    

In [107]:
#distributing into different clusters 


clusters_t_CS_m = []
clusters_t_CS_s = []
clusters_c_CS_m = []
clusters_c_CS_s = []
clusters_t_SC_m = []
clusters_t_SC_s = []
clusters_c_SC_m = []
clusters_c_SC_s = []

for node_elements in nodes_elements:
    labels = np.unique(df["class"].values[node_elements])
    if len(labels) == 1:
        # this is a pure node
        if labels[0] == "t-CS-m":
            clusters_t_CS_m.append(node_elements)
        elif labels[0] == "t-CS-s":
            clusters_t_CS_s.append(node_elements)
        elif labels[0] == "c-CS-m":
            clusters_c_CS_m.append(node_elements)
        elif labels[0] == "c-CS-s":
            clusters_c_CS_s.append(node_elements)
        elif labels[0] == "t-SC-m":
            clusters_t_SC_m.append(node_elements)
        elif labels[0] == "t-SC-s":
            clusters_t_SC_s.append(node_elements)
        elif labels[0] == "c-SC-m":
            clusters_c_SC_m.append(node_elements)
        elif labels[0] == "c-SC-s":
            clusters_c_SC_s.append(node_elements)
        

In [108]:
# Join the nodes together if connected by an edge
def join_clusters(clusters_class):
    output = clusters_class.copy()
    for iterate in range(5):
        for node in output:
            for ind, node2 in enumerate(output):
                if len(set(node).intersection(node2))>=1:
                    output.append(list(set(node).union(node2)))
                    output.pop(ind)
    return set(map(tuple,output))

In [118]:
f_clusters_t_CS_s = list(join_clusters(clusters_t_CS_s))
f_clusters_t_CS_m = list(join_clusters(clusters_t_CS_m))
f_clusters_c_CS_s = list(join_clusters(clusters_c_CS_s))
f_clusters_c_CS_m = list(join_clusters(clusters_c_CS_m))
f_clusters_t_SC_s = list(join_clusters(clusters_t_SC_s))
f_clusters_t_SC_m = list(join_clusters(clusters_t_SC_m))
f_clusters_c_SC_s = list(join_clusters(clusters_c_SC_s))
f_clusters_c_SC_m = list(join_clusters(clusters_c_SC_m))

In [132]:
# Remove clusters made of few elements (outliers!)
def remove_outliers(clusters):
    fin_clusters = []
    for ele in clusters:
        if len(ele) > 6:
            fin_clusters.append(ele)
    return fin_clusters

final_clusters_t_CS_m = remove_outliers(f_clusters_t_CS_m)
final_clusters_t_CS_s = remove_outliers(f_clusters_t_CS_s)
final_clusters_c_CS_m = remove_outliers(f_clusters_c_CS_m)
final_clusters_c_CS_s = remove_outliers(f_clusters_c_CS_s)
final_clusters_t_SC_m = remove_outliers(f_clusters_t_SC_m)
final_clusters_t_SC_s = remove_outliers(f_clusters_t_SC_s)
final_clusters_c_SC_m = remove_outliers(f_clusters_c_SC_m)
final_clusters_c_SC_s = remove_outliers(f_clusters_c_SC_s)

print("Number of clusters per class: ")
print("c_CS_s: ", len(final_clusters_c_CS_s))
print("c_CS_m: ", len(final_clusters_c_CS_m))
print("t_CS_s: ", len(final_clusters_t_CS_s))
print("t_CS_m: ", len(final_clusters_t_CS_m))
print("c_SC_s: ", len(final_clusters_c_SC_s))
print("c_SC_m: ", len(final_clusters_c_SC_m))
print("t_SC_s: ", len(final_clusters_t_SC_s))
print("t_SC_m: ", len(final_clusters_t_SC_m))

Number of clusters per class: 
c_CS_s:  6
c_CS_m:  5
t_CS_s:  4
t_CS_m:  7
c_SC_s:  4
c_SC_m:  6
t_SC_s:  7
t_SC_m:  6


In [133]:
# find the centers of each cluster
def find_cluster_centers(fin_cluster):
    mean_coord_centers = []
    for cluster1 in fin_cluster:
        mean_coord_centers.append(df.iloc[list(cluster1)].loc[:,"DYRK1A_N":"CaNA_N"].mean().values)
    return np.array(mean_coord_centers)

# extract the centers from each cluster by averaging the coordinates of all its elements
centers_c_CS_m = find_cluster_centers(final_clusters_c_CS_m)
centers_c_CS_s = find_cluster_centers(final_clusters_c_CS_s)
centers_c_SC_m = find_cluster_centers(final_clusters_c_SC_m)
centers_c_SC_s = find_cluster_centers(final_clusters_c_SC_s)
centers_t_CS_m = find_cluster_centers(final_clusters_t_CS_m)
centers_t_CS_s = find_cluster_centers(final_clusters_t_CS_s)
centers_t_SC_m = find_cluster_centers(final_clusters_t_SC_m)
centers_t_SC_s = find_cluster_centers(final_clusters_t_SC_s)

In [134]:
# Wilcoxon rank-sum test

# extract p-values
def p_values_comp(fin_cl_1,fin_cl_2):
    p_values = []

    for ind1, cluster1 in enumerate(fin_cl_1):
        for ind2, cluster2 in enumerate(fin_cl_2):
            for protein in proteins:
                try:
                    w, p = mannwhitneyu(x=df.iloc[list(cluster1)][protein], y=df.iloc[list(cluster2)][protein])
                    p_values.append([ind1, ind2, p])
                except:
                    p_values.append([ind1, ind2, 1])

    p_values = np.array(p_values)
    return p_values

#### Clusterwise comparison

In [135]:
# identify relevant proteins
def relevant_proteins(fin_cl_1,fin_cl_2,th):
    critical_proteins = []
    p_values = p_values_comp(fin_cl_1,fin_cl_2)
    for c1 in range(len(fin_cl_1)):
        for c2 in range(len(fin_cl_2)):
            i=0
            critical_prot_clust = []
            for elem in p_values:
                if elem[0]==c1 and elem[1]==c2:
                    if elem[2]< th:
                        critical_prot_clust.append(proteins[i])
                    i+=1
            critical_proteins.append(critical_prot_clust)

    list_set = map(set,critical_proteins)
    return set.intersection(*list_set)

In [150]:
THRESHOLD = 0.05
prot_m_s     = relevant_proteins(final_clusters_t_CS_s,final_clusters_t_CS_m,0.6)
prot_CS_SC   = relevant_proteins(final_clusters_c_CS_s,final_clusters_c_SC_s,THRESHOLD)
prot_CSm_SCm = relevant_proteins(final_clusters_t_CS_m,final_clusters_t_SC_m,THRESHOLD)

for threshold = 0.05 prot_m_s was not giving any protein so by experimentation the 0.6 fits good 


In [151]:
prot_m_s

{'PKCA_N',
 'PSD95_N',
 'RRP1_N',
 'S6_N',
 'pAKT_N',
 'pJNK_N',
 'pMTOR_N',
 'pP70S6_N',
 'pPKCAB_N',
 'pPKCG_N'}

### Results 
The relevant proteins

In [152]:
# The relevant proteins!
print("The proteins that are relevant for learning are:")
print(prot_CS_SC)
print("The proteins that are relevant for learning in down mice that are not affected by Memantine are:")
print(prot_CSm_SCm)
print("The proteins whose expression is affected by the Memantine drug are:")
print(prot_m_s)


The proteins that are relevant for learning are:
{'pPKCAB_N', 'pPKCG_N', 'pERK_N', 'SOD1_N', 'pGSK3B_N', 'PKCA_N', 'DYRK1A_N', 'pNUMB_N', 'ITSN1_N', 'CaNA_N'}
The proteins that are relevant for learning in down mice that are not affected by Memantine are:
{'pNR2A_N', 'pS6_N', 'pERK_N', 'Ubiquitin_N', 'SOD1_N', 'BRAF_N', 'CaNA_N', 'ARC_N', 'pMTOR_N'}
The proteins whose expression is affected by the Memantine drug are:
{'pPKCAB_N', 'pPKCG_N', 'PKCA_N', 'S6_N', 'pAKT_N', 'pJNK_N', 'PSD95_N', 'RRP1_N', 'pP70S6_N', 'pMTOR_N'}


#### Conclusions

The individuality of each mouse affects protein expression. Such individuality is far too great to have meaningful and interpretable results using standard statistical approach as the Wilcoxon rank-sum test. In order to partially group together individuals with similar genetic characteristics in an unsupervised way is necessary to obrain meaningful results. Thus, we need to cluster mice together to better understand how protein expression affect a subcluster

The use of the Mapper algorithm allowed for a more accurate identification of the most relevant proteins taking part in learning tasks. Furthemore, thanks to this powerful technique -- which do not require at all a pre-defined number of clusters -- it was possible to even characterise which proteins expression are affected by the Memantine compound. The Wilcoxon rank-sum test applied to portions of the Mapper graph enhanced the results significatively and allowed for a precise identifications of the relevant proteins.